# Introdução

No Aprendizado de Máquina e na estatística, a Classificação é o problema de identificar a qual de um conjunto de categorias (subpopulações) uma nova observação pertence, com base em um conjunto de dados de treinamento contendo observações cuja associação à categoria é conhecida. Exemplos de problemas de classificação são atribuir um determinado e-mail à classe "spam" ou "não spam" e atribuir um diagnóstico a um determinado paciente com base nas características observadas (sexo, pressão arterial, presença ou ausência de certos sintomas, etc.)

Neste notebook, usaremos o [Bank Marketing Dataset](https://drive.google.com/drive/folders/13a_SPs445s_IkJ4phKSHEAUJTfDuQ03O?usp=sharing) do Kaggle para construir um modelo para prever se alguém fará um depósito ou não, dependendo em alguns atributos, através do algoritmo de Árvore de Decisão. Após construir o modelo iremos avaliá-lo e ver como ele está performando para o nosso caso. 

Para começar vamos carregar algumas bibliotecas básicas como Pandas e NumPy e então fazer algumas configurações para algumas dessas bibliotecas.

In [ ]:
# Importar bibliotecas

## Bibliotecas básicas
import pandas as pd
import numpy as np
import warnings

## Visualização de dados
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar bibliotecas
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')

## Passo 1. Pré-processamento de dados

Antes de começarmos a criar nosso modelo, primeiro precisamos carregar e pré-processar. Esta etapa garante que nosso modelo receba bons dados para aprender, pois como dizem "um modelo é tão bom quanto seus dados". O pré-processamento dos dados será dividido em algumas etapas, conforme explicado a seguir.


### Carregando dados

Nesta primeira etapa, carregaremos nosso conjunto de dados que foi carregado do GitHub para facilitar o processo. A partir da documentação do conjunto de dados encontrada [aqui](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing), podemos ver abaixo a lista de colunas que temos em nossos dados:

Variáveis ​​de entrada (explicativas):
1. age (numérica)
2. job : tipo de trabalho (categorica: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : estado civil (categorica: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorica: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: tem crédito inadimplente? (categorica: 'no','yes','unknown')
6. housing: tem crédito habitação? (categorica: 'no','yes','unknown')
7. loan: tem empréstimo pessoal? (categorica: 'no','yes','unknown')
8. contact: tipo de comunicação de contato (categorica: 'cellular','telephone')
9. month: último contato mês do ano (categorica: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: último dia de contato da semana (categorica: 'mon','tue','wed','thu','fri')
11. duration: duração do último contato, em segundos (numérica). Observação importante: esse atributo afeta muito a variável target (por exemplo, se duração=0, então y='não'). No entanto, a duração não é conhecida antes de uma chamada ser realizada. Além disso, após o término da chamada, y é obviamente conhecido. Assim, essa entrada deve ser incluída apenas para fins de benchmark e deve ser descartada se a intenção for ter um modelo preditivo realista.
12. campaign: número de contatos realizados durante esta campanha e para este clientet (numeric, includes last contact)
13. pdays: número de dias que se passaram depois que o cliente foi contatado pela última vez em uma campanha anterior (numeric; 999 significa que o cliente não foi contatado anteriormente)
14. previous: número de contatos realizados antes desta campanha e para este cliente (numerica)
15. poutcome: resultado da campanha de marketing anterior (categorica: 'failure','nonexistent','success')

Variável de saída (target):
21. y. o cliente subscreveu um depósito a prazo? (binária: 'yes','no')

De acordo com a documentação do conjunto de dados, precisamos remover a coluna 'duration' porque, no caso real, a duração só é conhecida depois que a coluna do rótulo é conhecida. Esse problema pode ser considerado 'vazamento de dados' onde os preditores incluem dados que não estarão disponíveis no momento em que você fizer as previsões.


In [ ]:
# Carregar conjunto de dados
df_bank = pd.read_csv('https://raw.githubusercontent.com/rafiag/DTI2020/main/data/bank.csv')

# Descartar a coluna 'duration'
df_bank = df_bank.drop('duration', axis=1)

# print(df_bank.info())
print('Shape of dataframe:', df_bank.shape)
df_bank.head()

Shape of dataframe: (11162, 16)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,2,-1,0,unknown,yes


### Distribuiçao das classes

Outra coisa importante para se certificar antes de alimentar nossos dados no modelo é a distribuição de classe dos dados. No nosso caso onde a classe esperada é dividida em dois resultados, 'sim' e 'não', uma distribuição de classe de 50:50 pode ser considerada ideal.

In [ ]:
df_bank['deposit'].value_counts()

no     5873
yes    5289
Name: deposit, dtype: int64

Como podemos ver, nossa distribuição de classes é mais ou menos semelhante, não exatamente a distribuição 50:50, mas ainda é boa o suficiente.

###  Valores ausentes

Outra coisa a verificar antes de prosseguir são os valores ausentes. Em alguns casos, os dados podem ter valores ausentes em alguma coluna, isso pode ser causado por alguns motivos, como erro humano. Podemos usar a função `is_null()` do Pandas para verificar se há dados ausentes e, em seguida, usar a função `sum()` para ver o total de valores ausentes em cada coluna.


In [ ]:
df_bank.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

A partir do resultado, podemos ter certeza de que nossos dados não têm valor ausente e estão prontos para serem usados. No caso de você ter um valor ausente em seus dados, você pode resolvê-lo fazendo imputação ou apenas remover a coluna completamente, dependendo do seu caso.


### "Normalizar" dados numéricos

Em seguida, dimensionaremos nossos dados numéricos para evitar a presença de valores discrepantes que podem afetar significativamente nosso modelo. Usando a função `StandardScaler()` do sklearn, podemos dimensionar cada uma de nossas colunas que contêm dados numéricos. O dimensionamento será feito usando a fórmula abaixo:


<div align="center">$Z = \frac{X - U}{S}$</div>

*Onde:*

*$Z:$ valor "normalizado"*

*$X:$ valor original*

*$U:$ média dos dados*

*$S:$ desvio padrão dos dados*

In [ ]:
from sklearn.preprocessing import StandardScaler

# Copiando dataframe original
df_bank_ready = df_bank.copy()

scaler = StandardScaler()
num_cols = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']
df_bank_ready[num_cols] = scaler.fit_transform(df_bank_ready[num_cols])

df_bank_ready.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,deposit
0,1.491505,admin.,married,secondary,no,0.252525,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
1,1.239676,admin.,married,secondary,no,-0.459974,no,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
2,-0.019470,technician,married,secondary,no,-0.080160,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
3,1.155733,services,married,secondary,no,0.293762,yes,no,unknown,-1.265746,may,-0.554168,-0.481184,-0.36326,unknown,yes
4,1.071790,admin.,married,tertiary,no,-0.416876,no,no,unknown,-1.265746,may,-0.186785,-0.481184,-0.36326,unknown,yes


### Codificando valores categóricos

Assim como os dados numéricos, também precisamos pré-processar nossos dados categóricos de palavras para números para facilitar a compreensão do computador. Para fazer isso, usaremos `OneHotEncoder()` fornecido pelo sklearn. Basicamente, ele transformará uma coluna categórica disso:

| marital | housing |
|----------|---------|
| single   | yes     |
| divorced | no      |
| married  | no      |

...em algo assim...

| marital_single | marital_divorced | marital_married | housing_yes | housing_no |
|----------------|------------------|-----------------|-------------|------------|
| 1              | 0                | 0               | 1           | 0          |
| 0              | 1                | 0               | 0           | 1          |
| 0              | 0                | 1               | 0           | 1          |

Nesta célula de código, também codificaremos nossa coluna de rótulo substituindo 'sim' e 'não' por 1 e 0, respectivamente. Podemos fazer isso aplicando uma função lambda na coluna.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']

# Codificar dados categóricos
df_encoded = pd.DataFrame(encoder.fit_transform(df_bank_ready[cat_cols]))
df_encoded.columns = encoder.get_feature_names(cat_cols)

# Substituir dados categóricos por dados codificados
df_bank_ready = df_bank_ready.drop(cat_cols ,axis=1)
df_bank_ready = pd.concat([df_encoded, df_bank_ready], axis=1)

# Codificar valor de destino
df_bank_ready['deposit'] = df_bank_ready['deposit'].apply(lambda x: 1 if x == 'yes' else 0)

print('Shape of dataframe:', df_bank_ready.shape)
df_bank_ready.head()

Shape of dataframe: (11162, 51)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,poutcome_other,poutcome_success,poutcome_unknown,age,balance,day,campaign,pdays,previous,deposit
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.491505,0.252525,-1.265746,-0.554168,-0.481184,-0.36326,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.239676,-0.459974,-1.265746,-0.554168,-0.481184,-0.36326,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,-0.019470,-0.080160,-1.265746,-0.554168,-0.481184,-0.36326,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.155733,0.293762,-1.265746,-0.554168,-0.481184,-0.36326,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.071790,-0.416876,-1.265746,-0.186785,-0.481184,-0.36326,1


### Dividir conjunto de dados para treinamento e teste

Para finalizar nossas etapas de pré-processamento de dados, dividiremos nossos dados em dois conjuntos, treinamento e teste. Neste caso, como temos dados suficientes, dividiremos os dados com proporção de 70:30 para treinamento e teste, respectivamente. Isso resultará em nossos dados de treinamento com 7813 linhas e 3349 linhas para os dados de teste.


In [ ]:
# Select Features
feature = df_bank_ready.drop('deposit', axis=1)

# Select Target
target = df_bank_ready['deposit']

# Set Training and Testing Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature , target, 
                                                    shuffle = True, 
                                                    test_size=0.3, 
                                                    random_state=1)

# Show the Training and Testing Data
print('Shape of training feature:', X_train.shape)
print('Shape of testing feature:', X_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of training label:', y_test.shape)

Shape of training feature: (7813, 50)
Shape of testing feature: (3349, 50)
Shape of training label: (7813,)
Shape of training label: (3349,)


## Passo 2. Modelar

Depois de ter certeza de que nossos dados estão bons e prontos, podemos continuar construindo nosso modelo. Para avaliar nosso modelo, usaremos a matriz de confusão como base para a avaliação.

<div align='center'><img src='https://miro.medium.com/max/2102/1*fxiTNIgOyvAombPJx5KGeA.png' height='250'></div>
onde: TP = Verdadeiro Positivo; FP = Falso Positivo; TN = Verdadeiro Negativo; FN = Falso Negativo.

Usaremos 3 métricas abaixo para avaliar os modelos:

1. Acurácia: a proporção de resultados verdadeiros entre o número total de casos examinados.
<div align='center'>$Acuracia = \frac{TP+TN}{TP+TN+FP+FN}$</div>
2. Precisão: usado para calcular quanta proporção de todos os dados que foram previstos como positivos **foram** realmente positivos.
<div align='center'>$Precisão = \frac{TP}{TP+FP}$</div>
3. Recal: usado para calcular quanta proporção de positivos reais é classificada corretamente.
<div align='center'>$Recal = \frac{TP}{TP+FN}$</div>

Nesse caso, queremos nos concentrar no valor de recall de nosso modelo, porque em nosso problema devemos tentar prever o maior número possível de positivos reais. Porque uma classificação incorreta do cliente que **realmente** queria fazer um depósito pode significar perda de oportunidade/receita.

Abaixo vamos definir uma função auxiliar para avaliar o modelo treinado e com as métricas mencionadas acima e salvar a pontuação em uma variável.

In [ ]:
def evaluate_model(model, x_test, y_test):
    from sklearn import metrics

    # Prever dados de teste 
    y_pred = model.predict(x_test)

    # Calculando acurácia, precisão, recal
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred)
    rec = metrics.recall_score(y_test, y_pred)

    # Exibir matriz de confusão
    cm = metrics.confusion_matrix(y_test, y_pred)

    return {'acc': acc, 'prec': prec, 'rec': rec, 'cm': cm}

### Árvore de decisão

A árvore de decisão é um diagrama em forma de árvore usado para determinar um curso de ação. Cada ramo da árvore representa uma possível decisão, ocorrência ou reação.

<div align='center'><img src='https://raw.githubusercontent.com/rafiag/DTI2020/main/images/decision_tree.PNG' height='250'></div>



#### Construindo o modelo

In [ ]:
from sklearn import tree

# Construindo modelo de árvore de decisão 
dtc = tree.DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

#### Avaliação do modelo

In [ ]:
# Avaliar modelo
dtc_eval = evaluate_model(dtc, X_test, y_test)

# Resultado
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('Confusion Matrix:\n', dtc_eval['cm'])

Accuracy: 0.6324275903254702
Precision: 0.6162337662337662
Recall: 0.5972309628697294
Confusion Matrix:
 [[1169  591]
 [ 640  949]]


### Random Forest

Random forest or Random Decision Forest is a method that operates by constructing multiple decision trees during training phases. The decision of the majority of the trees is chosen as final decision.

<div align='center'><img src='https://raw.githubusercontent.com/rafiag/DTI2020/main/images/random_forest.PNG' height='250'></div>

Advantages:
* It can be used for both regression and classification tasks and that it’s easy to view the relative importance it assigns to the input features.
* It is also considered as a very handy and easy to use algorithm, because it’s default hyper-parameters often produce a good prediction result.

Disadvantages:
* Many trees can make the algorithm to slow and ineffective for real-time predictions. A more accurate prediction requires more trees, which results in a slower model.
* It is a predictive modelling tool and not a descriptive tool.

#### Building Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Building Random Forest model 
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

#### Model Evaluation

In [ ]:
# Evaluate Model
rf_eval = evaluate_model(rf, X_test, y_test)

# Print result
print('Accuracy:', rf_eval['acc'])
print('Precision:', rf_eval['prec'])
print('Recall:', rf_eval['rec'])
print('F1 Score:', rf_eval['f1'])
print('Cohens Kappa Score:', rf_eval['kappa'])
print('Area Under Curve:', rf_eval['auc'])
print('Confusion Matrix:\n', rf_eval['cm'])

Accuracy: 0.7205553067622034
Precision: 0.7488789237668162
Recall: 0.6254681647940075
F1 Score: 0.6816326530612244
Cohens Kappa Score: 0.43618595045335207
Area Under Curve: 0.781940492838887
Confusion Matrix:
 [[941 224]
 [400 668]]


### Naive Bayes

Naive Bayes is a simple technique for constructing classifiers: models that assign class labels to problem instances, represented as vectors of feature values, where the class labels are drawn from some finite set. There is not a single algorithm for training such classifiers, but a family of algorithms based on a common principle: all naive Bayes classifiers assume that the value of a particular feature is independent of the value of any other feature, given the class variable. Below are the Bayes theorem formula:

<div align="center">$P(C | A) = \frac{P(A|C) P(C)}{P(A)}$</div>

For example, given:
* A doctor knows that meningitis  causes  stiff neck 50% of the time
* Prior probability  of any patient  having  meningitis  is 1/50,000
* Prior probability  of any patient  having  stiff neck is 1/20

Then the probability of patient who have stiff neck to also have meningitis is:

<div align="center">$P(C | A) = \frac{P(A|C) P(C)}{P(A)} = \frac{0.5 * (1 / 50000)}{1 / 20} = 0.0002$</div>

#### Building Model

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Building Naive Bayes model 
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

#### Model Evaluation

In [ ]:
# Evaluate Model
nb_eval = evaluate_model(nb, X_test, y_test)

# Print result
print('Accuracy:', nb_eval['acc'])
print('Precision:', nb_eval['prec'])
print('Recall:', nb_eval['rec'])
print('F1 Score:', nb_eval['f1'])
print('Cohens Kappa Score:', nb_eval['kappa'])
print('Area Under Curve:', nb_eval['auc'])
print('Confusion Matrix:\n', nb_eval['cm'])

Accuracy: 0.6815942678011644
Precision: 0.7560975609756098
Recall: 0.4934456928838951
F1 Score: 0.5971671388101983
Cohens Kappa Score: 0.352622455965517
Area Under Curve: 0.7421999324878237
Confusion Matrix:
 [[995 170]
 [541 527]]


## Passo 3. Output

Temos o nosso modelo, e agora? Como cientista de dados, é importante ser capaz de desenvolver um modelo com boa reutilização. Nesta parte final vamos criar uma previsão com base em novos dados.


### Predição

Nesta etapa, vamos prever o resultado esperado de todas as linhas do nosso conjunto de dados e salvá-lo em um arquivo csv para facilitar o acesso no futuro.

In [ ]:
df_bank['deposit_prediction'] = dtc.predict(feature)
df_bank['deposit_prediction'] = df_bank['deposit_prediction'].apply(lambda x: 'yes' if x==0 else 'no')

# Salve o novo dataframe no arquivo csv
df_bank.to_csv('deposit_prediction.csv', index=False)

df_bank.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,deposit,deposit_prediction
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1,-1,0,unknown,yes,no
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1,-1,0,unknown,yes,no
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1,-1,0,unknown,yes,no
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,1,-1,0,unknown,yes,no
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,2,-1,0,unknown,yes,yes
5,42,management,single,tertiary,no,0,yes,yes,unknown,5,may,2,-1,0,unknown,yes,no
6,56,management,married,tertiary,no,830,yes,yes,unknown,6,may,1,-1,0,unknown,yes,yes
7,60,retired,divorced,secondary,no,545,yes,no,unknown,6,may,1,-1,0,unknown,yes,yes
8,37,technician,married,secondary,no,1,yes,no,unknown,6,may,1,-1,0,unknown,yes,no
9,28,services,single,secondary,no,5090,yes,no,unknown,6,may,3,-1,0,unknown,yes,no


## Passo 4. Conclusão

Para algo simples, podemos ver que nosso modelo se saiu decentemente na classificação dos dados. Mas ainda há algumas fraquezas, especialmente mostradas na métrica de recall, onde obtemos apenas cerca de 60%. Isso significa que nosso modelo é capaz de detectar apenas 60% dos clientes em potencial e perde os outros 40%.

# Referências

1. Telkom Digital Talent Incubator - Data Scientist Module 5 (Classification)
2. [Scikit-learn Documentation](https://scikit-learn.org/stable/index.html)
3. [The 5 Classification Evaluation metrics every Data Scientist must know](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)
4. [The Python Graph Gallery - Grouped Bar Plot](https://python-graph-gallery.com/11-grouped-barplot/)